### Load tensorflow

In [69]:
import tensorflow as tf

### Enable Eager Execution if you are using tensorflow 1.x

In [70]:
tf.enable_eager_execution()

### Collect Data

In [31]:
import numpy as np
import pandas as pd

In [32]:
data = pd.read_csv('./prices.csv')

### Check all columns in the dataset

In [33]:
data.shape

(851264, 7)

In [34]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [36]:
data.drop(['date','symbol'],axis=1, inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 5 columns):
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5)
memory usage: 32.5 MB


In [37]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [40]:
data1=data[:1000]

In [41]:
data1.shape

(1000, 5)

### Convert Float64 to Float32

In [44]:
data2=data1.astype('float32')

In [80]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
open      1000 non-null float32
close     1000 non-null float32
low       1000 non-null float32
high      1000 non-null float32
volume    1000 non-null float32
dtypes: float32(5)
memory usage: 19.6 KB


### Divide the data into train and test sets

In [49]:
X = data2.drop('close', axis=1)
y = data2.close

In [50]:
from sklearn.model_selection import train_test_split, GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((700, 4), (300, 4), (700,), (300,))

### Normalize Train and Test Data 

In [100]:
from sklearn.preprocessing import Normalizer
transformer = Normalizer()
X_train = transformer.fit_transform(X_train)

In [115]:
X_train

array([[1.95260873e-05, 1.92729567e-05, 1.95753055e-05, 1.00000000e+00],
       [3.25567339e-06, 3.24057760e-06, 3.36386051e-06, 1.00000000e+00],
       [1.67042970e-06, 1.65513768e-06, 1.70641090e-06, 1.00000000e+00],
       ...,
       [1.04472536e-04, 1.03533064e-04, 1.04582476e-04, 1.00000000e+00],
       [1.26646439e-04, 1.23368751e-04, 1.27374820e-04, 1.00000000e+00],
       [1.16344178e-04, 1.14913550e-04, 1.18731987e-04, 1.00000000e+00]],
      dtype=float32)

In [116]:
X_train.shape

(700, 4)

In [104]:
y_train=np.array(y_train)
y_train.shape

(700,)

## Building the graph in tensorflow

2.Define Weights and Bias

In [54]:
#initializing weights and Bias with Zero
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

3.Prediction

In [55]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

4.Loss (Cost) Function [Mean square error]

In [56]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [66]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Execute the Graph for 100 epochs and observe the loss

In [105]:
for i in range(100):
    
    w, b = train(X_train, y_train, w, b)
    print('Current Loss on iteration', i, loss(y_train, prediction(X_train, w, b)).numpy())

Current Loss on iteration 0 7653.4473
Current Loss on iteration 1 7333.088
Current Loss on iteration 2 7037.844
Current Loss on iteration 3 6765.747
Current Loss on iteration 4 6514.979
Current Loss on iteration 5 6283.8774
Current Loss on iteration 6 6070.8887
Current Loss on iteration 7 5874.6045
Current Loss on iteration 8 5693.702
Current Loss on iteration 9 5526.988
Current Loss on iteration 10 5373.337
Current Loss on iteration 11 5231.7393
Current Loss on iteration 12 5101.2397
Current Loss on iteration 13 4980.971
Current Loss on iteration 14 4870.135
Current Loss on iteration 15 4767.983
Current Loss on iteration 16 4673.8433
Current Loss on iteration 17 4587.083
Current Loss on iteration 18 4507.124
Current Loss on iteration 19 4433.438
Current Loss on iteration 20 4365.524
Current Loss on iteration 21 4302.937
Current Loss on iteration 22 4245.255
Current Loss on iteration 23 4192.0986
Current Loss on iteration 24 4143.106
Current Loss on iteration 25 4097.957
Current Loss o

### Get the shapes and values of W and b

In [106]:
#Check Weights and Bias
print('Weights:\n', w.numpy())


Weights:
 [[2.6111759e-03]
 [2.5876034e-03]
 [2.6312110e-03]
 [3.2731911e+01]]


In [107]:
print('Bias:\n',b.numpy())

Bias:
 [32.731922]


### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [178]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(1,input_shape=(4,)))
#model.add(tf.keras.layers.Dense(1))


#Comile the model
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)
model.compile(optimizer=sgd_optimizer, loss='mse')

In [179]:
X_train.reshape((700,4))

array([[1.95260873e-05, 1.92729567e-05, 1.95753055e-05, 1.00000000e+00],
       [3.25567339e-06, 3.24057760e-06, 3.36386051e-06, 1.00000000e+00],
       [1.67042970e-06, 1.65513768e-06, 1.70641090e-06, 1.00000000e+00],
       ...,
       [1.04472536e-04, 1.03533064e-04, 1.04582476e-04, 1.00000000e+00],
       [1.26646439e-04, 1.23368751e-04, 1.27374820e-04, 1.00000000e+00],
       [1.16344178e-04, 1.14913550e-04, 1.18731987e-04, 1.00000000e+00]],
      dtype=float32)

In [180]:
X_train.shape

(700, 4)

In [181]:
model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 1)                 5         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


### Execute the model

In [182]:
    model.fit(X_train, y_train, 
              validation_data=(X_test, y_test), 
              epochs=10,
              batch_size=32)

Train on 700 samples, validate on 300 samples
Epoch 1/10
700/700 [==============================] - 0s 104us/sample - loss: 4426.1712 - val_loss: 286764700494456000.0000
Epoch 2/10
700/700 [==============================] - 0s 27us/sample - loss: 3575.7347 - val_loss: 270299778581661408.0000
Epoch 3/10
700/700 [==============================] - 0s 36us/sample - loss: 3571.0956 - val_loss: 294916828310966080.0000
Epoch 4/10
700/700 [==============================] - 0s 28us/sample - loss: 3582.0766 - val_loss: 287846065226844736.0000
Epoch 5/10
700/700 [==============================] - 0s 31us/sample - loss: 3587.6879 - val_loss: 292819508506593856.0000
Epoch 6/10
700/700 [==============================] - 0s 33us/sample - loss: 3575.7824 - val_loss: 306720874076604544.0000
Epoch 7/10
700/700 [==============================] - 0s 34us/sample - loss: 3582.6741 - val_loss: 299036763205615808.0000
Epoch 8/10
700/700 [==============================] - 0s 30us/sample - loss: 3575.7857 - val

### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [292]:
i_df=pd.read_csv('Iris.csv')

In [293]:
i_df.shape

(150, 6)

In [294]:
i_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
Id               150 non-null int64
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), int64(1), object(1)
memory usage: 7.1+ KB


### Splitting the data into feature set and target set

In [295]:
X1 = i_df.drop(columns=['Species','Id'] ,axis=1)
y1 = i_df.Species

In [296]:
X1.shape

(150, 4)

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [229]:
from sklearn import preprocessing

In [230]:
#method1

In [297]:
le = preprocessing.LabelEncoder()

In [298]:
y1=le.fit_transform(i_df['Species'])

In [299]:
y1

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [300]:
y2=pd.get_dummies(y1).values

In [301]:
y2

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

#Method2
splitting data is pre req

### Divide the dataset into Training and test (70:30)

In [303]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.3, random_state=1)
X1_train.shape, X1_test.shape, y1_train.shape, y1_test.shape

((105, 4), (45, 4), (105,), (45,))

In [304]:
trainY1 = tf.keras.utils.to_categorical(y1_train, num_classes=3)
testY1 = tf.keras.utils.to_categorical(y1_test, num_classes=3)

In [305]:
trainY1

array([[0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0

In [306]:
testY1

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.]], dtype=float32)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [307]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Normalize the data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

#Add Dense Layer which provides 10 inputs/neron
model.add(tf.keras.layers.Dense(10, activation='relu'))


#Add Dense Layer which provides 8 inputs/neron
model.add(tf.keras.layers.Dense(8, activation='relu'))


#Add Dense Layer which provides 3 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3, activation='softmax'))

#Compile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])


In [308]:
X1_train=X1_train.values

In [309]:
X1_test=X1_test.values

In [310]:
X1_train

array([[7.7, 2.6, 6.9, 2.3],
       [5.7, 3.8, 1.7, 0.3],
       [5. , 3.6, 1.4, 0.2],
       [4.8, 3. , 1.4, 0.3],
       [5.2, 2.7, 3.9, 1.4],
       [5.1, 3.4, 1.5, 0.2],
       [5.5, 3.5, 1.3, 0.2],
       [7.7, 3.8, 6.7, 2.2],
       [6.9, 3.1, 5.4, 2.1],
       [7.3, 2.9, 6.3, 1.8],
       [6.4, 2.8, 5.6, 2.2],
       [6.2, 2.8, 4.8, 1.8],
       [6. , 3.4, 4.5, 1.6],
       [7.7, 2.8, 6.7, 2. ],
       [5.7, 3. , 4.2, 1.2],
       [4.8, 3.4, 1.6, 0.2],
       [5.7, 2.5, 5. , 2. ],
       [6.3, 2.7, 4.9, 1.8],
       [4.8, 3. , 1.4, 0.1],
       [4.7, 3.2, 1.3, 0.2],
       [6.5, 3. , 5.8, 2.2],
       [4.6, 3.4, 1.4, 0.3],
       [6.1, 3. , 4.9, 1.8],
       [6.5, 3.2, 5.1, 2. ],
       [6.7, 3.1, 4.4, 1.4],
       [5.7, 2.8, 4.5, 1.3],
       [6.7, 3.3, 5.7, 2.5],
       [6. , 3. , 4.8, 1.8],
       [5.1, 3.8, 1.6, 0.2],
       [6. , 2.2, 4. , 1. ],
       [6.4, 2.9, 4.3, 1.3],
       [6.5, 3. , 5.5, 1.8],
       [5. , 2.3, 3.3, 1. ],
       [6.3, 3.3, 6. , 2.5],
       [5.5, 2

In [311]:
X1_test

array([[5.8, 4. , 1.2, 0.2],
       [5.1, 2.5, 3. , 1.1],
       [6.6, 3. , 4.4, 1.4],
       [5.4, 3.9, 1.3, 0.4],
       [7.9, 3.8, 6.4, 2. ],
       [6.3, 3.3, 4.7, 1.6],
       [6.9, 3.1, 5.1, 2.3],
       [5.1, 3.8, 1.9, 0.4],
       [4.7, 3.2, 1.6, 0.2],
       [6.9, 3.2, 5.7, 2.3],
       [5.6, 2.7, 4.2, 1.3],
       [5.4, 3.9, 1.7, 0.4],
       [7.1, 3. , 5.9, 2.1],
       [6.4, 3.2, 4.5, 1.5],
       [6. , 2.9, 4.5, 1.5],
       [4.4, 3.2, 1.3, 0.2],
       [5.8, 2.6, 4. , 1.2],
       [5.6, 3. , 4.5, 1.5],
       [5.4, 3.4, 1.5, 0.4],
       [5. , 3.2, 1.2, 0.2],
       [5.5, 2.6, 4.4, 1.2],
       [5.4, 3. , 4.5, 1.5],
       [6.7, 3. , 5. , 1.7],
       [5. , 3.5, 1.3, 0.3],
       [7.2, 3.2, 6. , 1.8],
       [5.7, 2.8, 4.1, 1.3],
       [5.5, 4.2, 1.4, 0.2],
       [5.1, 3.8, 1.5, 0.3],
       [6.1, 2.8, 4.7, 1.2],
       [6.3, 2.5, 5. , 1.9],
       [6.1, 3. , 4.6, 1.4],
       [7.7, 3. , 6.1, 2.3],
       [5.6, 2.5, 3.9, 1.1],
       [6.4, 2.8, 5.6, 2.1],
       [5.8, 2

### Fitting the model and predicting 

# METHOD1 - #using - tf.keras.utils.to_categorical values

In [332]:
#using - tf.keras.utils.to_categorical values
model.fit(X1_train, trainY1, 
 validation_data=(X1_test, testY1), 
 epochs=100,
 batch_size=20)

Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 0s 76us/sample - loss: 0.1997 - acc: 0.9333 - val_loss: 0.2511 - val_acc: 0.8667
Epoch 2/100
105/105 [==============================] - 0s 85us/sample - loss: 0.2644 - acc: 0.8952 - val_loss: 0.2496 - val_acc: 0.8667
Epoch 3/100
105/105 [==============================] - 0s 85us/sample - loss: 0.3214 - acc: 0.8381 - val_loss: 0.2471 - val_acc: 0.8667
Epoch 4/100
105/105 [==============================] - 0s 85us/sample - loss: 0.2165 - acc: 0.9143 - val_loss: 0.2455 - val_acc: 0.8889
Epoch 5/100
105/105 [==============================] - 0s 76us/sample - loss: 0.1779 - acc: 0.9619 - val_loss: 0.2446 - val_acc: 0.8889
Epoch 6/100
105/105 [==============================] - 0s 76us/sample - loss: 0.1908 - acc: 0.9524 - val_loss: 0.2440 - val_acc: 0.8889
Epoch 7/100
105/105 [==============================] - 0s 85us/sample - loss: 0.2168 - acc: 0.9238 - val_loss: 0.2428 - val_acc: 0.8889
Epo

105/105 [==============================] - 0s 86us/sample - loss: 0.1553 - acc: 0.9619 - val_loss: 0.2204 - val_acc: 0.8889
Epoch 61/100
105/105 [==============================] - 0s 95us/sample - loss: 0.1919 - acc: 0.9333 - val_loss: 0.2243 - val_acc: 0.8889
Epoch 62/100
105/105 [==============================] - 0s 95us/sample - loss: 0.2046 - acc: 0.9048 - val_loss: 0.2306 - val_acc: 0.8667
Epoch 63/100
105/105 [==============================] - 0s 76us/sample - loss: 0.2411 - acc: 0.8762 - val_loss: 0.2270 - val_acc: 0.8667
Epoch 64/100
105/105 [==============================] - 0s 104us/sample - loss: 0.1758 - acc: 0.9429 - val_loss: 0.2303 - val_acc: 0.8667
Epoch 65/100
105/105 [==============================] - 0s 85us/sample - loss: 0.1647 - acc: 0.9619 - val_loss: 0.2213 - val_acc: 0.8889
Epoch 66/100
105/105 [==============================] - 0s 86us/sample - loss: 0.1977 - acc: 0.9048 - val_loss: 0.2186 - val_acc: 0.8889
Epoch 67/100
105/105 [==============================]

### Report Accuracy of the predicted values

In [333]:
#using - tf.keras.utils.to_categorical values
model.evaluate(X1_test, testY1)

45/45 [==============================] - 0s 66us/sample - loss: 0.2020 - acc: 0.9556


[0.2019571410285102, 0.95555556]

# METHOD2 - #using - tf.keras.utils.to_categorical values

In [319]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y2, test_size=0.3, random_state=1)
X1_train.shape, X1_test.shape, y1_train.shape, y1_test.shape

((105, 4), (45, 4), (105, 3), (45, 3))

In [320]:
X1_train=X1_train.values

In [321]:
X1_test=X1_test.values

## Fitting the model and predicting

In [330]:
#using - tf.keras.utils.to_categorical values
model.fit(X1_train, y1_train, 
 validation_data=(X1_test, y1_test), 
 epochs=100,
 batch_size=20)

Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 0s 85us/sample - loss: 0.3199 - acc: 0.8762 - val_loss: 0.3498 - val_acc: 0.8444
Epoch 2/100
105/105 [==============================] - 0s 85us/sample - loss: 0.2850 - acc: 0.9238 - val_loss: 0.3503 - val_acc: 0.8222
Epoch 3/100
105/105 [==============================] - 0s 85us/sample - loss: 0.2918 - acc: 0.9048 - val_loss: 0.3490 - val_acc: 0.8222
Epoch 4/100
105/105 [==============================] - 0s 95us/sample - loss: 0.3594 - acc: 0.8476 - val_loss: 0.3485 - val_acc: 0.8222
Epoch 5/100
105/105 [==============================] - 0s 76us/sample - loss: 0.2794 - acc: 0.8857 - val_loss: 0.3464 - val_acc: 0.8222
Epoch 6/100
105/105 [==============================] - 0s 76us/sample - loss: 0.3127 - acc: 0.8667 - val_loss: 0.3455 - val_acc: 0.8222
Epoch 7/100
105/105 [==============================] - 0s 66us/sample - loss: 0.3006 - acc: 0.8952 - val_loss: 0.3459 - val_acc: 0.8222
Epo

105/105 [==============================] - 0s 85us/sample - loss: 0.2671 - acc: 0.8857 - val_loss: 0.2849 - val_acc: 0.8889
Epoch 61/100
105/105 [==============================] - 0s 95us/sample - loss: 0.3078 - acc: 0.8857 - val_loss: 0.2839 - val_acc: 0.8889
Epoch 62/100
105/105 [==============================] - 0s 104us/sample - loss: 0.2196 - acc: 0.9429 - val_loss: 0.2838 - val_acc: 0.8889
Epoch 63/100
105/105 [==============================] - 0s 85us/sample - loss: 0.2103 - acc: 0.9333 - val_loss: 0.2841 - val_acc: 0.8889
Epoch 64/100
105/105 [==============================] - 0s 95us/sample - loss: 0.2111 - acc: 0.9429 - val_loss: 0.2842 - val_acc: 0.8667
Epoch 65/100
105/105 [==============================] - 0s 95us/sample - loss: 0.2587 - acc: 0.9048 - val_loss: 0.2804 - val_acc: 0.8889
Epoch 66/100
105/105 [==============================] - 0s 95us/sample - loss: 0.2269 - acc: 0.9238 - val_loss: 0.2780 - val_acc: 0.8889
Epoch 67/100
105/105 [==============================]

## Report Accuracy of the predicted values


In [331]:
# Using Label Encoder categorical values
model.evaluate(X1_test, y1_test)

45/45 [==============================] - 0s 44us/sample - loss: 0.2518 - acc: 0.8667


[0.25184061262342666, 0.8666667]